In [22]:
import os
import xarray as xr
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rasterio as rio
from rasterio.mask import mask
from rasterio.plot import show
from rasterio.plot import show_hist
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.windows import Window
from rasterio.merge import merge
from rasterio import features
import geopandas as gpd
#from shapely.geometry import box
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import fiona
from fiona.crs import from_epsg
#import pycrs
#import pathlib
import glob
from osgeo import gdal, ogr
import sys
import pprint
import csv
import re
import pandas as pd
import rioxarray

import netCDF4
from datetime import datetime
import earthpy as et


In [93]:
#extract pixel value from coordinate location

def extractValueSM(nc_dir, lev, lat, long):
    files = np.array(os.listdir(nc_dir))
    netcdfs = pd.Series(files).str.contains('.nc')
    files = files[netcdfs]
    output = []
  
    for filename in files: 
        data = xr.open_dataset(nc_dir + filename)
        val = data.SoilMoist_tavg[lev, int((np.abs(data.Y - lat)).argmin()),int((np.abs(data.X - long)).argmin())]
        output.append(float(val.values))
    return np.asarray(output)

def extractValueNDVI(tiff_dir, long, lat):
    files = np.array(os.listdir(tiff_dir))
    tiffs = pd.Series(files).str.contains('.tiff')
    files = files[tiffs]
    result = []
    
    for filename in files: 
        data = rio.open(tiff_dir + filename)
        val = data.sample([long,lat])
        for val in data.sample([(long,lat)]):
            vals = ((val-100)/100)
        result.append(*vals)    
    return np.asarray(result)
        
def extractValueCHIRPS(tiff_dir, long, lat):
    files = np.array(os.listdir(tiff_dir))
    tiffs = pd.Series(files).str.contains('.tif')
    files = files[tiffs]
    result = []
    
    for filename in files: 
        data = rio.open(tiff_dir + filename)
        val = data.sample([long,lat])
        for val in data.sample([(long,lat)]):
            vals = val
        result.append(*vals)    
    return np.asarray(result)
        
def stackTiffs(tiff_dir):
    files = np.array(os.listdir(tiff_dir))
    tiffs = pd.Series(files).str.contains('.tif')
    files = files[tiffs]
    tiflist = []
    
    for filename in files:
        data = xr.open_rasterio(tiff_dir + filename)
        tiflist.append(data)
        stacked = xr.concat(tiflist, dim=pd.Index(dekads, name = 'time'))
    return stacked
        
    

# Create dataframe for one pixel

In [ ]:
soil_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/FLDAS/'
ndvi_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/eMODIS_NDVI/CropsMask/'
chirps_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/CHIRPS/'
sm_level = 0
long = 38.87
lat = 4.21


In [ ]:
soil = extractValueSM(soil_dir, sm_level, lat, long) #coordinates in Y, X, specify soil level
ndvi = extractValueNDVI(ndvi_dir, long, lat) #coordinates in X,Y
chirps = extractValueCHIRPS(chirps_dir, long, lat)

In [59]:
#Establish date period
begin = pd.datetime(2002,7,1)
end = pd.datetime(2019,4,30)

dtrange = pd.date_range(begin, end)

In [ ]:
#Create Dataframe for one pixel time series
daily = pd.DataFrame({'Soil Moisture': soil}, index=dtrange) #create dataframe

d = daily.index.day - np.clip((daily.index.day-1) // 10, 0, 2)*10 - 1 #convert daily SM values to dekadal
date = daily.index.values - np.array(d, dtype="timedelta64[D]")
TS = daily.groupby(date).mean()
TS.index.names = ['Date']
TS['NDVI'] = ndvi
TS['CHIRPS'] = chirps

In [ ]:
#export to csv 
TS.to_csv(r'/home/rgreen/tana-spin/rgreen/DroughtEDM/TS_3887_421.csv') 

In [62]:
#create list of dekads as datetime objects
days = list(range(0,6148)) #length of dtrange
daysDF = pd.DataFrame({'Days': days}, index=dtrange)
d = daysDF.index.day - np.clip((daysDF.index.day-1) // 10, 0, 2)*10 - 1 #convert daily SM values to dekadal
dekaddates = daysDF.index.values - np.array(d, dtype="timedelta64[D]")
dekads = daysDF.groupby(dekaddates).mean()
dekads = dekads.index


DatetimeIndex(['2002-07-01', '2002-07-11', '2002-07-21', '2002-08-01',
               '2002-08-11', '2002-08-21', '2002-09-01', '2002-09-11',
               '2002-09-21', '2002-10-01',
               ...
               '2019-01-21', '2019-02-01', '2019-02-11', '2019-02-21',
               '2019-03-01', '2019-03-11', '2019-03-21', '2019-04-01',
               '2019-04-11', '2019-04-21'],
              dtype='datetime64[ns]', length=606, freq=None)

# Dataframe with all pixel values

In [ ]:
sm1 = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/FLDAS/LIS_HIST_200207010000.d01.nc'
sm2 = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/FLDAS/LIS_HIST_200207020000.d01.nc'
sm3 = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/FLDAS/LIS_HIST_200207030000.d01.nc'
sm4 = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/FLDAS/LIS_HIST_200207040000.d01.nc'
sm_data = xr.open_dataset(sm)
smlev1 = np.array(sm_data.variables['SoilMoist_tavg'][0,:,:])
#.rio.to_raster("SM20020701.tif", tiled=True, compress="LZMA")


In [ ]:
ndvi1 = xr.open_rasterio('/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/eMODIS_NDVI/EastAfrica/data.2002.071_mask.tiff')
ndvi2 = xr.open_rasterio('/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/eMODIS_NDVI/EastAfrica/data.2002.072_mask.tiff')
#ndvi3 = xr.open_dataset('/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/eMODIS_NDVI/EastAfrica/data.2002.073_mask.tiff')
#ndvi4 = xr.open_dataset('/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/eMODIS_NDVI/EastAfrica/data.2002.081_mask.tiff')

#ndvi_list = ['ndvi1', 'ndvi2', 'ndvi3', 'ndvi4']

ndvi1.name = 'day1'
ndvi2.name = 'day2'
#ndvi3.name = 'day3'
#ndvi4.name = 'day4'

d1 = xr.merge([ndvi1, ndvi2])

d1.day1.values

twodays = xr.concat([ndvi1, ndvi2], dim=pd.Index([20020701, 20020702], name = 'time'))


In [87]:
pd.Index([dekads[:4]], name = 'time')

Index([[2002-07-01 00:00:00, 2002-07-11 00:00:00, 2002-07-21 00:00:00, 2002-08-01 00:00:00]], dtype='object', name='time')

In [69]:
pd.Index([20020701, 20020702], name = 'time')

Int64Index([20020701, 20020702], dtype='int64', name='time')

In [ ]:
##does not work need to fix
#function to extract dekads from filename
def datesFromfile(directory):
    files = os.listdir(directory)
    dates = []
    for filename in files:
        file = re.search(r'\d{4}.\d{3}', filename)
        print(file.group())
    return dates
#dates.sort()



In [72]:
ndvi_files = glob.glob('/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/eMODIS_NDVI/EastAfrica/*.tiff')

In [91]:
xr.merge([xr.open_rasterio(f) for f in ndvi_files[:4]])

ValueError: unable to convert unnamed DataArray to a Dataset without providing an explicit name

In [88]:
ndvi_merge =  xr.concat([xr.open_rasterio(f) for f in ndvi_files[:4]]), dim = 'time')

SyntaxError: invalid syntax (<ipython-input-88-04a63fbcd068>, line 1)

In [ ]:
ndvi_stack = xr.concat

In [ ]:
stackTiffs('/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/eMODIS_NDVI/EastAfrica/')

In [ ]:
#Stack tiffs


with rio.open(ndvi_files[0]) as src0:
    meta = src0.meta

meta.update(count=len(ndvi_files))

with rio.open('ndvi_stack.tiff', 'w', **meta) as dst:
    for id, layer in enumerate(ndvi_files, start=1):
        with rio.open(layer) as src1:
            dst.write_band(id, src1.read(1))

In [20]:
stack = rasterio.open('/home/rgreen/tana-spin/rgreen/DroughtEDM/ndvi_stack.tiff')
stack.meta

{'driver': 'GTiff',
 'dtype': 'uint8',
 'nodata': 0.0,
 'width': 9946,
 'height': 11190,
 'count': 606,
 'crs': CRS.from_epsg(4326),
 'transform': Affine(0.0024129999999999998, 0.0, 27.999396000000022,
        0.0, -0.002413, 15.001320649815025)}

# Random code

In [ ]:
fldas = os.listdir('/home/rgreen/tana-crunch/rgreen/DroughtEDM/Data/FLDAS/200207/')
dates = []
for filename in fldas:
    file = re.search(r'\d{4}.\d{3}', filename)
    dates.append(file.group())
    #print(file.group)

dates.sort()
print(dates)

In [ ]:
dates2 = pd.to_datetime(pd.Series(dates), format ='%Y%m%d')
pytesmo.timedate.dekad.check_dekad(dates2[1])
#s = pd.Series(['dates2'], name = "dates")
#df = pd.DataFrame('Dates':dates)

#df.insert(1, 'M', [str(x) for x in range (1,4) for i in range(10)])

#df

In [ ]:
n=10
final = [output[i * n:(i + 1) * n] for i in range((len(output) + n - 1) // n )]  
print (final)
len(final)

In [ ]:
test3 = np.mean(outputa[:(len(outputa)//10)*10].reshape(-1,10), axis=1)
test3

In [ ]:
data = pd.read_csv('TS_3887_421.csv.csv')
data.insert(2,'SM', dates)

In [ ]:
f = netCDF4.Dataset('/home/rgreen/tana-crunch/rgreen/DroughtEDM/Data/FLDAS/200207/LIS_HIST_200207010000.d01.nc')

print(f.variables.keys())

In [ ]:
allfiles = glob.glob('/home/rgreen/tana-crunch/rgreen/DroughtEDM/Data/FLDAS/200207/*.nc')

In [ ]:
#data200207 = xr.open_mfdataset('/home/rgreen/tana-crunch/rgreen/DroughtEDM/Data/FLDAS/200207/*.nc', concat_dim=[..., None, ...])

In [ ]:
data1 = '/home/rgreen/tana-crunch/rgreen/DroughtEDM/Data/FLDAS/200207/LIS_HIST_200207010000.d01.nc'
data2 = '/home/rgreen/tana-crunch/rgreen/DroughtEDM/Data/FLDAS/200207/LIS_HIST_200207020000.d01.nc'
data27 = '/home/rgreen/tana-crunch/rgreen/DroughtEDM/Data/FLDAS/FLDAS_NOAH01_C_GL_M.A200207.001.nc' #global monthly
DS1 = xr.open_dataset(data1)
DS2 = xr.open_dataset(data2)
DS27 = xr.open_dataset(data27)
DS1.SoilMoist_tavg[0,400:405,300:305]
#xidx = (np.abs(DS1.X - -17.83)).argmin()
#yidx = (np.abs(DS1.Y - long)).argmin()
DS1.Evap_tavg[1,0]

In [ ]:
DS1.SoilMoist_tavg[0,568,442]

In [ ]:
lat, lon = np.array(DS1.variables['Y'][:]), np.array(DS1.variables['X'][:])


In [ ]:
print(find_nearest(lat, 4))

In [ ]:
lat, lon = DS1.variables['Y'], DS1.variables['X']
latvals = lat[:]; lonvals = lon[:]

lat = np.array(DS1.variables['Y'][:])
sm = np.array(DS1.variables['SoilMoist_tavg'][0,:,:]) #change first value to indicate soil level, 0 = level 1
#iy_min, ix_min = getclosest_ij(latvals, lonvals, 4, 38)
lat.shape

In [ ]:
#iy_min, ix_min = getclosest_ij(latvals, lonvals, 4, 38)

dist_sq = (latvals - 4)**2 + (lonvals - 38)**2
idy, idx = np.where(dist_sq==dist_sq.min())
idy[0]
idx[0]

In [ ]:
i = np.abs(lonvals - 38).argmin()
j = np.abs(latvals -4).argmin()
DS1.SoilMoist_tavg[0,j,i]

In [ ]:
ax = plt.axes(projection=ccrs.PlateCarree())
ax.coastlines() 
sm1.plot()
plt.show()

In [ ]:
sm1.cartopy_crs

In [ ]:
sm1_ras = rio.to_raster(sm1)

In [ ]:
sm_ras = rio.open('netcdf:/home/rgreen/tana-crunch/rgreen/DroughtEDM/Data/FLDAS/200207/LIS_HIST_200207010000.d01.nc:SoilMoist_tavg')

In [ ]:
#clip SM images to cropzones boundaries

with fiona.open('/home/rgreen/tana-crunch/rgreen/DroughtEDM/Data/cropzones/cropzones.shp', "r") as shapefile:
    features = [feature["geometry"] for feature in shapefile]

src = rio.open('/home/rgreen/tana-crunch/rgreen/DroughtEDM/Data/eMODIS_NDVI/EastAfrica/data.2002.071_mask.tiff')

    
show((src,1))
ax = plt.gca()
patches = [PolygonPatch(feature, edgecolor="red", facecolor="none", linewidth=2) for feature in features]
ax.add_collection(mpl.collections.PatchCollection(patches, match_original=True))


In [ ]:
    #val = data.sample([x,y])
       #output = []
        
      #  for result in data.sample([(x, y)]): 
           # with openoutput.append(*((result-100)/100))
          # output =  ((result-100)/100)
       # return output
        
       # return result
            
            #output = (*((result-100)/100))
           # print
    
        #return return np.asarray(output)
        
        #np.asarray(output)
# with rio.open(tiff_dir + filename) as data:
         #   output.append(data.sample([(x, y)]))

In [ ]:
data = rio.open('/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/eMODIS_NDVI/CropsMask/data.2002.071_mask_crops.tiff')
for val in data.sample([(38.87, 4.21)]): print(val)


In [48]:
class0=np.zeros((3,4))
class1=np.ones((3,4))
class2=np.ones((3,4))*2
class3=np.ones((3,4))*3
class_list=[class0,class1,class2,class3]
class_list

[array([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]]), array([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]]), array([[2., 2., 2., 2.],
        [2., 2., 2., 2.],
        [2., 2., 2., 2.]]), array([[3., 3., 3., 3.],
        [3., 3., 3., 3.],
        [3., 3., 3., 3.]])]

In [52]:
class_list[1,0,0]

TypeError: list indices must be integers or slices, not tuple

In [51]:
lenlist=[x.shape[1,0,0] for x in class_list]
lenlist

TypeError: tuple indices must be integers or slices, not tuple

In [33]:
m = np.concatenate(class_list, axis=0)
m

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [2., 2., 2., 2.],
       [2., 2., 2., 2.],
       [2., 2., 2., 2.],
       [2., 2., 2., 2.],
       [2., 2., 2., 2.],
       [3., 3., 3., 3.],
       [3., 3., 3., 3.]])

In [57]:
m[4,1]

1.0

### notes: 
Either need to aggregate by pixels, time series with all pixels or sample by pixels 
